In [74]:
#ENUM
import random
class Sentiment:
    NEGATIVE="NEGATIVE"
    NEUTRAL="NEUTRAL"
    POSITIVE="POSITIVE"

class Review:
    def __init__(self,text,score):
        self.text=text
        self.score=score
        self.sentiment=self.get_sentiment()
    def get_sentiment(self):
        if self.score<=2:
            return Sentiment.NEGATIVE
        elif self.score==3:
            return Sentiment.NEUTRAL
        else:
            return Sentiment.POSITIVE
class ReviewContainer:
    def __init__(self,reviews):
        self.reviews=reviews
    def get_text(self):
        return [x.text for x in self.reviews]
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
        
    def evenly_distribute(self):
        negative=list(filter(lambda x:x.sentiment==Sentiment.NEGATIVE,self.reviews))
        positive=list(filter(lambda x:x.sentiment==Sentiment.POSITIVE,self.reviews))
        # print(negative[0].text)
        # print(len(negative))
        # print(len(positive))
        #length of negative is less than length of positive,we will reduce the length of positive
        positive_shrunk=positive[:len(negative)]
        self.reviews=positive_shrunk+negative
        random.shuffle(self.reviews)
        
        
             

In [75]:
import json
file_name='./Books_small_10000.json'
reviews=[]
with open(file_name) as f:
    for line in f:
        review=json.loads(line)
        # print(review['reviewText'])
        reviews.append(Review(review['reviewText'],review['overall']))
        
# reviews[5]
# reviews[5][1]
reviews[5].text
        
        



'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

In [76]:
from sklearn.model_selection import train_test_split
training,test=train_test_split(reviews,test_size=0.33,random_state=42)
# len(training)
train_container=ReviewContainer(training)
test_container=ReviewContainer(test)





In [77]:
train_container.evenly_distribute()
# print(training[0].text)
# print(training[0].sentiment)
# print(training[0].score)
train_x=train_container.get_text()
train_y=train_container.get_sentiment()
test_container.evenly_distribute()
test_x=test_container.get_text()
test_y=test_container.get_sentiment()
# print(train_y.count(Sentiment.POSITIVE))
# print(train_y.count(Sentiment.NEGATIVE))
# train_y.count(Sentiment.NEGATIVE)
# train_x[0]
# train_y[0]


In [78]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer  #term frequency inverse document frequency
vectorizer=TfidfVectorizer()
# vectorizer=CountVectorizer()
# vectorizer.fit_transform(train_x)
vectorizer.fit(train_x)
train_x_vectors=vectorizer.transform(train_x)
test_x_vectors=vectorizer.transform(test_x)


In [79]:
#classification
# from sklearn.svm import SVC
from sklearn import svm
clf_svm=svm.SVC(kernel='linear')
clf_svm.fit(train_x_vectors,train_y)
# test_x[0]
clf_svm.predict(test_x_vectors[0])






array(['NEGATIVE'], dtype='<U8')

In [80]:
# #DECISION TREE ,IMPLEMENTING TEXT CLASSIFICATION USING DECISION TREES
from sklearn.tree import DecisionTreeClassifier
clf_dec=DecisionTreeClassifier()
clf_dec.fit(train_x_vectors,train_y)
clf_dec.predict(test_x_vectors[0])




array(['NEGATIVE'], dtype='<U8')

In [81]:
#IMPLEMENTING NAIVE BAYES WITH NAIVE_bAYES CLASSIFIER
from sklearn.naive_bayes import GaussianNB
clf_gnb=DecisionTreeClassifier()

clf_gnb.fit(train_x_vectors,train_y)
clf_gnb.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

In [82]:
#IMPLEMENTING TEXT CLASSIFICATION USING LOGISTIC REGRESSION
from sklearn.linear_model import LogisticRegression
clf_log=LogisticRegression()
clf_log.fit(train_x_vectors,train_y)
clf_log.predict(test_x_vectors[0])


array(['NEGATIVE'], dtype='<U8')

In [83]:
#EVALUATION PHASE,CALCULATING THE ACCURACY(SCORE)
print(clf_svm.score(test_x_vectors,test_y))
print(clf_dec.score(test_x_vectors,test_y))
print(clf_gnb.score(test_x_vectors,test_y))
print(clf_log.score(test_x_vectors,test_y))



0.8076923076923077
0.6153846153846154
0.6466346153846154
0.8052884615384616


In [84]:
#F1 Scores
from sklearn.metrics import f1_score
f1_score(test_y,clf_svm.predict(test_x_vectors),average=None,labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE])
# f1_score(test_y,clf_dec.predict(test_x_vectors),average=None,labels=[Sentiment.POSITIVE,Sentiment.NEUTRAL,Sentiment.NEGATIVE])

array([0.80582524, 0.80952381])

In [85]:
train_y.count(Sentiment.POSITIVE)
#out of 670 we have 552 positive,so our model is heavily biased towards the positive


436

In [86]:
train_y.count(Sentiment.NEGATIVE)

436

In [87]:
test_y.count(Sentiment.NEGATIVE)

208

In [88]:
test_set=['very good book','bad book do not boy','horrible waste of time']
new_test=vectorizer.transform(test_set)
clf_svm.predict(new_test)

array(['POSITIVE', 'NEGATIVE', 'NEGATIVE'], dtype='<U8')

In [89]:
#Tuning model,improving accuracy using GridsearchCV
from sklearn.model_selection import GridSearchCV
parameters={'kernel':('linear','rbf'),'C':(1,4,8,16,32)}
svc=svm.SVC()
clf=GridSearchCV(svc,parameters,cv=5)
clf.fit(train_x_vectors,train_y)



GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': (1, 4, 8, 16, 32), 'kernel': ('linear', 'rbf')})

In [91]:
#saving model
import pickle
with open('./sentiment_classifier.pkl','wb') as f:
    pickle.dump(clf,f)

In [93]:
#load model
with open('./sentiment_classifier.pkl','rb') as f:
    loaded_clf=pickle.load(f)

In [96]:
print(test_x[0])
loaded_clf.predict(test_x_vectors[0])

Not something I would read again.. Sorry but the stories just lack substance .. It's basically &#34;porn&#34; in written form.. Only thing missing is the cheesy music..


array(['NEGATIVE'], dtype='<U8')